In [ ]:
import sys
import importlib
sys.path.append(r'../../pykneer')
sys.path.append(r'../pykneer')
import pykneer_io as io
importlib.reload(io)
import preprocessing_for_nb as prep
importlib.reload(prep)
import segmentation_sa_for_nb as segm
importlib.reload(segm)

import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline

from preparing_images import prepare_image_and_list

In [ ]:
input_file_name           = ".\image_list_preprocessing.txt"
n_of_cores                = 1 # change the number of cores according to your computer
intensity_standardization = 1

# Image information
image_data = io.load_image_data_preprocessing(input_file_name)

ref = sitk.ReadImage('../../reference/newsubject/reference.mha')

# Spatial standardization
# Step 0: Read the dicom images
prep.read_dicom_stack(image_data, n_of_cores)
# Save image header
prep.print_dicom_header(image_data, n_of_cores)    # save to *.txt
    
# Step 1: Change orientation to RAI
#prep.axial_to_sagittal(image_data, n_of_cores)
prep.orientation_to_rai(image_data, n_of_cores)

# Step 2: Change laterality if knee is right
prep.flip_rl(image_data, n_of_cores)
# Step 3: Set image origin to (0,0,0)
prep.origin_to_zero(image_data, n_of_cores)    # Images are saved as *_orig.mha and they are anonymized

# 4. Correct magnetic field inhomogeneities
# Magnetic fields inhomogeneities create grey shades on images. This correction removes these shades. 
# This is the longest step of the processing. It can take up to 15-20 min on a standard PC or laptop
if intensity_standardization == 1:
    prep.field_correction(image_data, n_of_cores)
    
# 5. Rescale intensities to [0 100]
if intensity_standardization == 1:
    prep.rescale_to_range(image_data, n_of_cores) 
    
# 6. Edge preserving smoothing
if intensity_standardization == 1:
    prep.edge_preserving_smoothing(image_data, n_of_cores) 

In [ ]:
input_file_name = "./image_list_newsubject.txt"
modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
n_of_cores      = 1

image_data = io.load_image_data_segmentation(modality, input_file_name)
segm.prepare_reference(image_data)
### NOTE!!! reference_fv is dummy!!!

# Segment Bone
# 1. Register image to reference
segm.register_bone_to_reference(image_data, n_of_cores)
# 2. Invert transformations
segm.invert_bone_transformations(image_data, n_of_cores)
# 3. Warp reference mask to moving image
segm.warp_bone_mask(image_data, n_of_cores)

In [ ]:
input_file_name = "./image_list_longitudinal.txt"
modality        = "longitudinal" # use "newsubject", "longitudinal", or "multimodal" 
n_of_cores      = 1

image_data = io.load_image_data_segmentation(modality, input_file_name)
segm.prepare_reference(image_data)

segm.register_bone_to_reference(image_data, n_of_cores)
segm.invert_bone_transformations(image_data, n_of_cores)
segm.warp_bone_mask(image_data, n_of_cores)

segm.register_vessel_to_reference(image_data, n_of_cores)
segm.invert_vessel_transformations(image_data, n_of_cores)
segm.warp_vessel_mask(image_data, n_of_cores)